In [1]:
import os 

In [2]:
%pwd 

'/Users/nathanwandji/Documents/GitHub/MLFLOW_project_for_SG/research'

In [3]:
os.chdir('../')

In [4]:
%pwd 

'/Users/nathanwandji/Documents/GitHub/MLFLOW_project_for_SG'

# 1) MLFlow configuration 

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/NathanMacktravis/MLFLOW_project_for_SG.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="NathanMacktravis"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d68f124f0b12bb4ce768e9a0c8337350ad76d72c"

Pour configurer l’environnement pour que MLflow envoie automatiquement toutes les expériences vers DagsHub, avec authentification sécurisée: 

* export MLFLOW_TRACKING_URI=https://dagshub.com/NathanMacktravis/MLFLOW_project_for_SG.mlflow

* export MLFLOW_TRACKING_USERNAME=userName 

* export MLFLOW_TRACKING_PASSWORD=Token


# 2) Initialize Class entity step (--> src/entity/cong_entity.py)

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

# 3) Configs manager step (--> src/config/configuration.py)

In [9]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/NathanMacktravis/MLFLOW_project_for_SG.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

# 4) Component definition step (--> src/components/name_of_the_step)

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

/Users/nathanwandji/Documents/GitHub/MLFLOW_project_for_SG/.env/lib/python3.12/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [16]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

    def log_into_mlflow(self):
        # Définir l’URI de tracking MLflow (DagsHub)
        mlflow.set_tracking_uri(self.config.mlflow_uri)

        with mlflow.start_run():
            # Log des paramètres
            mlflow.log_params(self.config.all_params)

            # Log des métriques
            mlflow.log_metrics(
                {
                    "loss": self.score[0],
                    "accuracy": self.score[1]
                }
            )

            # Log du modèle en tant qu’artefact (SANS Model Registry)
            mlflow.keras.log_model(
                self.model,
                artifact_path="model",
                registered_model_name="VGG16Model"
            )


# 4) Pipeline step (src/pipeline/stage_number)

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 563ms/step - accuracy: 1.0000 - loss: 0.0047


2026/01/26 09:29:27 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2026/01/26 09:29:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
